# import Statements

In [84]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint


In [85]:
df = pd.read_csv("intrusion_detection/data.csv")
# df = df.rename(columns={"left": "label"})


In [86]:
df.head()

,duration,service,src_bytes,dst_bytes,hot,num_failed_logins,num_compromised,num_root,num_file_creations,num_access_files,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack
0,0,20,491,0,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,45,146,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,50,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,dos
3,0,25,232,8153,0,0,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,25,199,420,0,0,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


# Training and Validation data split

In [2]:
def train_validate_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size*len(df))
        
    indices=df.index.tolist()
    validate_indices = random.sample(population=indices,k=test_size)
    
    validate_df=df.loc[validate_indices]
    train_df=df.drop(validate_indices)
    return train_df,validate_df

In [90]:
# random.seed(0)
# train_df,validate_df=train_validate_split(df,test_size=0.2)

In [136]:
data = df.values
data=data[:,:-1]
label=df.values
label=label[:,-1]
# validate_df.head()
name_list=list(df)

# print(data.shape,label.shape)

mean_list=[]
new_values=data.transpose()
# print(new_values.shape,data.shape)
# for x in new_values:
#     mn=np.mean(x,axis=0)
#     mean_list.append(mn)

# print(mean_list)
X = df.loc[:,'duration':'dst_host_srv_rerror_rate']

# for i in range(len(data)):
X.loc[:,'duration':'dst_host_srv_rerror_rate']=(X.loc[:,'duration':'dst_host_srv_rerror_rate'] - X.loc[:,'duration':'dst_host_srv_rerror_rate'].mean())/X.loc[:,'duration':'dst_host_srv_rerror_rate'].std()
    
# print(X.shape)
# Q=X.transpose()
# print(Q.shape)
# Q=np.matmul(X,Q)
# print(Q.shape)
covarience=np.cov(np.array(X.T))
# print(covarience.shape)

eigen_values,eigen_vectors=np.linalg.eig(covarience)
# print(eigen_values)

max_val=[]
for i in range(len(eigen_values)):
    temp=eigen_values[i]/sum(eigen_values)
    max_val.append((temp,i))
    
max_val=sorted(max_val)
max_val.reverse()
# print(max_val)
newdata_list=[]
temp=0

for x in max_val:
    if temp >= (.9):
        break
    temp+=x[0]
    newdata_list.append(x[1])

# print(type(data),type(new_data))
new_data=eigen_vectors[:,newdata_list]    
# print()
new_data=np.dot(data,new_data)
print(new_data)

[[1.0088979558034532 -0.2720021948827624 0.029598648975205442 ...
  0.280442341499135 -0.057627211901477245 0.4771033963369577]
 [-0.4958353196679695 1.120903950154108 0.21195672938555726 ...
  0.21439008388723804 -0.06168700824797629 0.06104956626688599]
 [-3.802320432310103 -1.2480931250055818 0.058765030183155154 ...
  0.11971321618535924 -0.0009648465820513712 -0.27839586697524316]
 ...
 [-0.24896782131296935 -0.33617935388851966 0.12014285587019619 ...
  0.3363274075073062 0.02335845954290435 -0.9219415786100246]
 [-3.728407337900388 -1.326753843893294 -0.00657661335271564 ...
  0.1285149580720921 -0.013027276739955332 0.36068198015592695]
 [1.0552810608634218 -0.38181057150433895 -0.01454478284592672 ...
  0.3827533968290799 -0.046476109049515275 0.35920655002133595]]
